In [2]:
from Bio import motifs
import os
import sys
import numpy as np

sys.path.append('/home/hxcai/cell_type_specific_CRE')
from utils import *

In [3]:
def save_to_fasta(sequences, filename):
    """
    将一组DNA序列保存为FASTA格式文件。
    
    :param sequences: 一个包含seq的列表
    :param filename: 保存的文件名，例如 "sequences.fasta"
    """
    with open(filename, 'w') as file:
        for seq_id, sequence in enumerate(sequences):
            file.write(f">{seq_id}\n")
            file.write(f"{sequence}\n")

df = pd.read_csv('../../data/GosaiMPRA/GosaiMPRA_total.csv')
save_to_fasta(df['seq'], 'GosaiMPRA_total.fa')

In [ ]:
subprocess.run('split -l 100000 GosaiMPRA_total.fa GosaiMPRA_part_', shell=True)
subprocess.run('ls GosaiMPRA_part_* > file_list.txt', shell=True)
subprocess.run('cat file_list.txt | parallel -j 30 "fimo --skip-matched-sequence --no-pgc --max-stored-scores 10000000 JASPAR2024_CORE_vertebrates_non-redundant_pfms_meme.txt {} > {}_fimo_output.txt"', shell=True)
subprocess.run('cat GosaiMPRA_part_*_fimo_output.txt > fimo_output.txt', shell=True)

In [4]:
# # # cmd = 'fimo --oc fimo_output JASPAR2024_CORE_vertebrates_non-redundant_pfms_meme.txt Gosai.fa'
# subprocess.run('fimo --skip-matched-sequence --no-pgc --max-stored-scores 10000000 JASPAR2024_CORE_vertebrates_non-redundant_pfms_meme.txt GosaiMPRA_total.fa > fimo_output.txt', shell=True)

Using motif +MA0004.1 of width 6.
Using motif -MA0004.1 of width 6.
Using motif +MA0069.1 of width 14.
Using motif -MA0069.1 of width 14.


In [3]:
fimo_df = pd.read_csv('fimo_output_total.txt', sep='\t')

/tmp/ipykernel_2568832/769640985.py:1: DtypeWarning: Columns (2,3,4,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  fimo_df = pd.read_csv('fimo_output_total.txt', sep='\t')


In [4]:
fimo_df

,motif_id,motif_alt_id,sequence_name,start,stop,strand,score,p-value,q-value,matched_sequence
0,MA0069.1,PAX6,141,93,106,+,13.4,0.000012,NaN,NaN
1,MA0069.1,PAX6,142,93,106,+,16.2778,0.000001,NaN,NaN
2,MA0069.1,PAX6,328,107,120,+,12.7667,0.000018,NaN,NaN
3,MA0069.1,PAX6,329,107,120,+,12.7667,0.000018,NaN,NaN
4,MA0069.1,PAX6,395,59,72,-,11.5667,0.000038,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
40057113,MA0619.2,LIN54,839791,79,85,+,11.5873,0.000097,NaN,NaN
40057114,MA0619.2,LIN54,839809,61,67,-,11.5873,0.000097,NaN,NaN
40057115,MA0619.2,LIN54,839829,97,103,+,11.5873,0.000097,NaN,NaN
40057116,MA0619.2,LIN54,839838,153,159,+,11.5873,0.000097,NaN,NaN


In [5]:
counts_df = fimo_df.groupby(['sequence_name', 'motif_alt_id']).size().unstack(fill_value=0)
counts_df

motif_alt_id,ARGFX,ARNT2,ASCL1,ATF2,ATF3,ATF4,ATF6,ATF7,ATOH7,Alx1,...,Zbtb2,Zfp335,Zfp809,Zfp961,Zfx,Zic1::Zic2,Zic2,Zic3,Znf423,motif_alt_id
sequence_name,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
99991,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0


In [9]:
# 创建包含所有索引的完整索引列表
complete_index = pd.Index(range(839851))

# 重新索引 counts_df，以包含所有索引
counts_df = counts_df.reindex(complete_index, fill_value=0)
counts_df

motif_alt_id,ARGFX,ARNT2,ASCL1,ATF2,ATF3,ATF4,ATF6,ATF7,ATOH7,Alx1,...,Zbtb2,Zfp335,Zfp809,Zfp961,Zfx,Zic1::Zic2,Zic2,Zic3,Znf423,motif_alt_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839848,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
839849,0,0,0,1,2,0,0,2,0,0,...,1,0,0,0,0,0,0,0,0,0
839850,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
839851,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
TF_names = load_txt('TF_names.txt')
for TF_name in TF_names:
    if TF_name not in counts_df.columns:
        counts_df.loc[:, TF_name] = 0

In [11]:
counts_df

motif_alt_id,ARGFX,ARNT2,ASCL1,ATF2,ATF3,ATF4,ATF6,ATF7,ATOH7,Alx1,...,PITX1,Twist2,EMX1,Nobox,DMRTA2,TBP,SP5,Shox2,GSX1,TLX2
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839848,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
839849,0,0,0,1,2,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
839850,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
839851,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
counts_df.to_csv('GosaiMPRA_TF_counts_total.csv', index=False)

# 后面是纯python方法，很慢

In [42]:
file_path = "JASPAR2024_CORE_vertebrates_non-redundant_pfms_jaspar.txt"
with open(file_path) as handle:
    motif_list = motifs.parse(handle, "jaspar")

# file_path = "JASPAR2024_CORE_vertebrates_non-redundant_pfms_meme.txt"
# with open(file_path) as handle:
#     motif_list = motifs.parse(handle, "MINIMAL")

print(len(motif_list))
print(len(set(motif_list)))
for motif in motif_list:
    print(motif.name)

879
879
Arnt
PAX6
RORA
RXRA::VDR
REL
RELA
NR1H2::RXRA
Znf423
NFIC::TLX1
ZNF354C
Pou5f1::Sox2
EWSR1-FLI1
Arid3a
INSM1
RARA::RXRA
PLAG1
Tfcp2l1
DUX4
Sox6
ESR2
SREBF1
SREBF2
KLF5
Atf1
Creb3l2
FOXG1
Foxj2
BHLHE41
ELF4
FOXI1
FOXL1
HNF1B
HSF1
IRF9
JDP2
MEF2B
MLX
MSC
MYF6
NEUROG2
Nr2e1
OLIG2
PAX4
Rxra
SRF
TBXT
TBX20
TFAP4
ZIC1
PHOX2A
PROP1
VENTX
Nr2F6
RARA
RARA
BCL6B
EGR2
GLIS2
GLIS3
KLF16
ZNF143
E2F7
HSF2
HSF4
MEF2D
MEIS2
MYBL1
MYBL2
NFKB1
PAX3
PKNOX2
POU3F2
POU3F3
POU3F4
POU5F1B
PROX1
SMAD3
TGIF1
TGIF2
RUNX2
MGA
TBX15
TBX1
TBX4
TBX5
TEAD3
TFAP2B
TFAP2C
Ascl2
HES7
HEY1
OLIG1
OLIG3
CEBPG
FOXB1
FOXC1
FOXO4
FOXO6
FOXP3
RXRB
RXRG
Rarb
Rarb
Rarg
TP63
TP53
GMEB2
MTF1
E2F1
SOX21
Sox1
TFAP2A
ZNF24
ZSCAN4
FOSB::JUN
FOSL1::JUN
FOSB::JUNB
IRF5
TCF7L1
ATOH7
FERD3L
FOXN3
HES6
IRF6
NR2F1
NR2F6
RXRB
RXRG
SMAD5
TFAP4
TGIF2LX
TGIF2LY
ZBTB32
ZNF135
ZNF136
ZNF382
ZNF460
ZNF528
HOXA5
HOXD3
NEUROG1
ZNF707
ZNF8
Foxn1
ELK1::HOXA1
HOXB2::ELK1
POU2F1::SOX2
ZFP14
ZNF418
ZNF669
SP1
RFX1
SP2
BHLHA15
SP4
KLF14
KLF12
RFX

In [44]:
motif_names = list(set([motif.name for motif in motif_list]))
print(len(motif_names))

828


In [45]:
save_txt('TF_names.txt', motif_names)

In [5]:
def to_array(pwm):
    pwm = np.array([pwm[nuc] for nuc in 'ACGT'])
    return pwm

In [7]:
def scan_pwm(onehot, pwm):
    pwm = pwm.T
    pwm_len = len(pwm)
    scores = np.zeros(len(onehot) - pwm_len + 1)
    for i in range(len(scores)):
        window = onehot[i:i + pwm_len]
        scores[i] = np.sum(window * pwm)
    return scores

In [8]:
onehot = str2onehot(df.at[1, 'seq'])
pwm = to_array(motif_list[0].pwm)

print(onehot.shape, pwm.shape)
scores = scan_pwm(onehot, pwm)
print(scores.shape)

(200, 4) (4, 11)
(190,)


In [9]:
score_matrix = np.zeros((len(df), len(motif_list)))
for j, motif in enumerate(motif_list):
    for i, seq in tqdm(enumerate(df['seq']), total=len(df)):
        onehot = str2onehot(seq)
        # pwm = to_array(motif.pwm)
        # scores = scan_pwm(onehot, pwm)
        # score_matrix[i, j] = np.max(scores)

        # rpssm = motif.pssm.reverse_complement()
        results = motif.pssm.search(seq, threshold=0)
        scores = [result[1] for result in results]
        if scores:
            score_matrix[i, j] = np.max(scores)

 16%|█▌        | 11946/76168 [00:01<00:08, 7914.41it/s]

In [ ]:
seq = df.at[1, 'seq']
onehot = str2onehot(seq)
pwm = to_array(motif_list[0].pwm)

print(onehot.shape, pwm.shape)
scores = scan_pwm(onehot, pwm)
print(scores.shape)

In [ ]:
seq = "ATGCGTACGTAGCTAGCTAGCTAGCGTACGTAGCTAG"

# 扫描序列并输出结果
for motif in motif_list:
    scores = motif.pssm.calculate(seq)
    print(f"Motif: {motif.name}")
    print(f"Scores: {scores}")

Motif: RUNX1
Scores: [-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf]
Motif: TFAP2A
Scores: [-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf]
Motif: Arnt
Scores: [-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf]
Motif: Ahr::Arnt
Scores: [-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf]
Motif: Ar
Scores: [-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf]
Motif: Tbxt
Scores: [-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf]
Motif: Pax5
Scores: [-inf -inf -inf -i

In [ ]:
motif.pwm

{'A': (0.04113110539845758,
  0.9048843187660668,
  0.007712082262210797,
  0.910025706940874,
  0.6889460154241646,
  0.9254498714652957,
  0.570694087403599),
 'C': (0.11825192802056556,
  0.0,
  0.02570694087403599,
  0.0,
  0.0,
  0.007712082262210797,
  0.005141388174807198),
 'G': (0.04627249357326478,
  0.005141388174807198,
  0.005141388174807198,
  0.012853470437017995,
  0.0,
  0.05141388174807198,
  0.11311053984575835),
 'T': (0.794344473007712,
  0.08997429305912596,
  0.961439588688946,
  0.07712082262210797,
  0.3110539845758355,
  0.015424164524421594,
  0.3110539845758355)}

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [ ]:
df = pd.read_csv('../../data/GosaiMPRA/GosaiMPRA_designed.csv', sep='\t')

In [ ]:
df

,seq,origin,K562,HepG2,SKNSH,HepG2_b,K562_b,SKNSH_b
0,TCGAAACAGGTATACCGCCAGCAATCCAATCTCTACCGGTGTACAC...,Simulated_Annealing,-0.311911,2.196081,-0.917639,1,0,0
1,GCTAAGGTCCTTATTGAATTTTGGTTAATCATTAACTACCCGTCAT...,Simulated_Annealing,-1.110302,0.880661,-1.097117,0,0,0
2,TGGTCCATTCCTGGGAATTTGCCGCCTGTACCTGCAAGGTTATACA...,Simulated_Annealing,-0.545083,2.169025,-1.135171,1,0,0
3,TCGGGAGATGTCAACACCTGTTGAGCTATTATGCCCTAGCCTGGTT...,Simulated_Annealing,-0.769610,2.042634,-0.703300,1,0,0
4,GACTTTGAACTCTGGACAGCAGAACAAATCACGGGAACCCTGTCAT...,Simulated_Annealing,-0.654328,2.037279,-1.243780,1,0,0
...,...,...,...,...,...,...,...,...
76163,CTCGGAGTTCAAGAAAATCATCTCACACGGCCGTCTCAGACCCTTC...,control,-0.315267,0.591628,0.068697,0,0,0
76164,GCAATCCTACCCAGACAGATGCAGACAGCGACCTGGTGGGGGATGT...,control,-0.134821,-0.359782,-0.022281,0,0,0
76165,CCTATGCTGCCTCCTATGCCCTTGTCATCATTGCCATTTTTGAGCT...,control,0.646673,0.489900,0.462366,0,0,0
76166,AGCCACAACAAAACAGCCAGTTAATGTGGAAACAAGGTCGACAACT...,control,-0.061676,-0.014411,0.239693,0,0,0


In [ ]:
score_matrix = np.zeros((len(df), len(motif_list)))
for i, seq in tqdm(enumerate(df['seq']), total=len(df)):
    for j, motif in enumerate(motif_list):
        score_matrix[i, j] = np.max(motif.pssm.calculate(seq))
        # rpssm = motif.pssm.reverse_complement()
        # results = motif.pssm.search(seq, threshold=0)
        # scores = [result[1] for result in results]
        # if scores:
        #     score_matrix[i, j] = np.max(scores)

  0%|          | 0/76168 [00:00<?, ?it/s]

  3%|▎         | 2341/76168 [06:21<3:21:17,  6.11it/s]

In [ ]:
score_df = pd.DataFrame
names = [motif.name for motif in motif_list]